# Amazon Batchjob 생성 예시

## JONSL 파일 생성 및 S3 업로드

In [ ]:
import json
import uuid
import boto3
from datetime import datetime

# S3 클라이언트 생성
s3_client = boto3.client(
    's3',
    region_name='us-west-2'
)

# 버킷 이름
bucket_name = 'knowledgebase-bedrock-agent-gsmoon'
bucket_prefix = 'batch_inference'

# 현재 시간을 이용한 파일 이름 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'batch_records_{timestamp}.jsonl'

# 로컬 파일 경로
local_file_path = f'/tmp/{file_name}'

# JSONL 파일 생성
with open(local_file_path, 'w', encoding='utf-8') as file:
    for _ in range(100):
        # 11자리 영숫자로 된 고유 ID 생성
        record_id = uuid.uuid4().hex[:11]
        
        # 레코드 생성
        record = {
            "recordId": record_id,
            "modelInput": {"text": "Tell me the fun story of a cat in korean"}
        }
        
        # JSONL 형식으로 파일에 쓰기
        file.write(json.dumps(record, ensure_ascii=False) + '\n')

# S3에 파일 업로드
try:
    s3_client.upload_file(local_file_path, bucket_name, bucket_prefix + '/' + file_name)
    print(f"파일이 성공적으로 업로드되었습니다: s3://{bucket_name}/{bucket_prefix}/{file_name}")
except Exception as e:
    print(f"업로드 중 오류가 발생했습니다: {e}")

## 기본 정보 세팅

In [ ]:
import sagemaker
import boto3

# SageMaker 세션 생성
sagemaker_session = sagemaker.Session()

# 기본 IAM 역할 ARN 가져오기
role = sagemaker.get_execution_role()



## Job 생성

In [ ]:
import boto3
import json
from typing import List, Dict
from datetime import datetime as dt

def run_batch_inference(
    input_s3_uri: str,
    output_s3_uri: str,
    role_arn: str,
    job_name: str,
    model_id: str
) -> Dict:
    """
    Run batch inference using Nova models with batch API
    """
    bedrock = boto3.client('bedrock', region_name='us-west-2')
    today_str = dt.today().strftime("%y%m%d")
    job_name = f"nova-pro-batch-{job_name}-{today_str}"
    try:
        response = bedrock.create_model_invocation_job(
            jobName=job_name,
            roleArn=role_arn,
            modelId=model_id,
            inputDataConfig={
                "s3InputDataConfig": {
                    "s3Uri": input_s3_uri,
                    "s3InputFormat": "JSONL"  # JSON에서 JSONL로 변경
                }
            },
            outputDataConfig={
                "s3OutputDataConfig": {
                    "s3Uri": output_s3_uri
                }
            },
            timeoutDurationInHours=168
        )
        return response
    except Exception as e:
        print(f"Error during batch inference: {str(e)}")
        raise
    
if __name__ == "__main__":
    job_name=f"job-batch-inference-test-02"
    bucket_output_prefix = "batch_inference_output"
    input_s3_uri = f"s3://{bucket_name}/{bucket_prefix}/{file_name}"
    output_s3_uri = f"s3://{bucket_name}/{bucket_output_prefix}/"
    role_arn = role
    # print(f"job_{num}")
    print(job_name)
    print(f"input_s3_uri: {input_s3_uri}")
    print(f"output_s3_uri: {output_s3_uri}")
    result = run_batch_inference(
        input_s3_uri=input_s3_uri,
        output_s3_uri=output_s3_uri,
        role_arn=role_arn,
        job_name=job_name,
        model_id="us.amazon.nova-pro-v1:0"
    )

## Job 생성 결과

![batch_job_console.png](img/batch_job_console.png)